In [ ]:
from board_labeler import get_files_in_dir

filesDir = "labeledConstSize/"

paths = get_files_in_dir(filesDir)

In [ ]:
import cv2
import re
import os
from sklearn.preprocessing import StandardScaler
from board_labeler import dictClasses
from board_labeler import dctInv

scaler = StandardScaler()

stringLen = len(filesDir)
m = re.compile(r"(\/)([a-zA-Z_]+)(\/)")

X = [cv2.imread(x) for x in paths]
y = [m.findall(x)[0][1] for x in paths]

y = [[dictClasses[x]] for x in y]

print(f"{len(X)} {len(y)}")

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.asanyarray(X)
y = np.asanyarray(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
import shutil

train_path = 'train_data/'
test_path = 'test_data/'

if os.path.exists(train_path) and os.path.isdir(train_path):
    shutil.rmtree(train_path)
os.mkdir(train_path)

if os.path.exists(test_path) and os.path.isdir(test_path):
    shutil.rmtree(test_path)
os.mkdir(test_path)

for key, value in dictClasses.items():
    os.mkdir(f"{train_path}[{value}]")
    os.mkdir(f"{test_path}[{value}]")

for i in range(len(X_train)):
    cv2.imwrite(f"{train_path}{y_train[i]}/{i}.png", X_train[i])
    
for i in range(len(X_test)):
    cv2.imwrite(f"{test_path}{y_test[i]}/{i}.png", X_test[i])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

imgShape = X_train[0].shape
num_classes = len(dictClasses)
print(num_classes)

inner_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=X_train[0].shape,
    classes=num_classes,
)
inner_model.trainable = False
model = keras.Sequential([
    keras.Input(shape=X_train[0].shape),
    layers.RandomContrast(0.5),
    layers.RandomTranslation(0.3, 0.3),
    layers.RandomZoom(0.3, 0.3, fill_mode="constant", interpolation="bilinear"),
    layers.RandomRotation(0.5, fill_mode="constant", interpolation="bilinear"),
    inner_model,
    layers.Flatten(),
    layers.Dense(200, activation='relu'),
    layers.Dense(100, activation='relu'),
    #Dense(50, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax'),
])

model.compile(
    optimizer='adam',   
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.fit(
    X_train,
    keras.utils.to_categorical(y_train),
    epochs=20,
    batch_size=128,
)

In [ ]:
score = model.evaluate(X_test, keras.utils.to_categorical(y_test), verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
model.save("models/secondmodel")